# Szyfrowanie + kompresja
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Konfiguracja tunelu VPN/sprawozdanie.md](https://github.com/Gombek7/techniki-poufnosci/blob/main/Konfiguracja%20tunelu%20VPN/sprawozdanie.md)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - własny algrytm szyfrowania/deszyfrowania

Wybrany algorytm to szyfr Cezara.

### Szyfrator i deszyfrator

Najpierw został zdefiniowany alfabet, czyli zestaw znaków obsługiwanych przez nasz algorytm.

In [9]:
const ALPHABET = "AĄBCĆDEĘFGHIJKLŁMNŃOÓPRSŚTUVWXYZŹŻaąbcćdeęfghijklłmnńoóprsśtuvwxyzźż\b\t\n\f\r !\"%'()*+,-.:?"
// const ALPHABET = "AĄBCĆDEĘFGHIJKLŁMNŃOÓPRSŚTUVWXYZŹŻaąbcćdeęfghijklłmnńoóprsśtuvwxyzźż\b\t\n\f\r !\"#$%&'()*+,-./0123456789:;<=>?@[\\]^`{|}~"

Następnie została napisana funkcja szyfrująca `encrypt`.

In [10]:
function encrypt(input: string, key: number): string
{
    const output: string[] = [];
    const n = ALPHABET.length; //liczba znaków w alfabecie
    for (const char of input) {
        const a = ALPHABET.indexOf(char); //indeks znaku do zaszyfrowania
        const c = (a + key) % n ;         //indeks zaszyfrowanego znaku
        output.push(ALPHABET[c]) ;        //zamiana indeksu na znak i wpisanie do wyjścia
    }
    return output.join("");
}

Funkcja deszyfrująca `decrypt` jest podobna, z tą różnicą, że klucz jest odejmowany od indeksu znaku.

In [63]:
function decrypt(input: string, key: number):string
{
    const output: string[] = []
    const n = ALPHABET.length //liczba znaków w alfabecie
    for(const letter of input)
    {
        const c = ALPHABET.indexOf(letter) //indeks zaszyfrowanego znaku
        const a = (c + n - key) % n        //indeks odszyfrowanego znaku
        output.push(ALPHABET.at(a))        //zamiana indeksu na znak i wpisanie do wyjścia
    }
    return output.join("")
}

Poniżej znajduje się krótki test szyfru. Podany tekst jest zaszyfrowywany losowym kluczem. Następnie odszyfrowywany prawidłowym i nieprawidłowym kluczem.

In [89]:
import { randomIntegerBetween } from 'jsr:@std/random';

const input = "Cześć! To jest tekst oryginalny!";
const key = randomIntegerBetween(0, 2 * ALPHABET.length);
const invalidKey = key + randomIntegerBetween(1, Math.ceil(ALPHABET.length / 2));

const encrypted = encrypt(input, key);
const decryptedInvalid = decrypt(encrypted, invalidKey);
const decryptedValid = decrypt(encrypted, key);

Deno.jupyter.md`| **Tekst oryginalny** | ${input} |
| --- | --- |
| Prawidłowy klucz | ${key} |
| Nieprawidłowy klucz | ${invalidKey} |
| **Tekst zaszyfrowany** | **${JSON.stringify(encrypted)}** |
| Tekst odszyfrowany z nieprawidłowym kluczem | ${decryptedInvalid} |
| **Tekst odszyfrowany z prawidłowym kluczem** | **${decryptedValid}** |
`

| **Tekst oryginalny** | Hello! This is the input! |
| --- | --- |
| Prawidłowy klucz | 223 |
| Nieprawidłowy klucz | 237 |
| **Tekst zaszyfrowany** | **"CŻęęjżźŃcćmźćmźńcŻźćhlońż"** |
| Tekst odszyfrowany z nieprawidłowym kluczem | {UaadutIYZgtZgtiYUtZcęjiu |
| **Tekst odszyfrowany z prawidłowym kluczem** | **Hello! This is the input!** |


## Łamanie metodą brute force

Przypuśćmy, że atakujący zna alfabet, w którym została napisana wiadomość. Może go uzyskać poprzez przechwycenie kilku zaszyfrowanych wiadomości. Atakujący może z łatwością znaleźć wszystkie możliwe odszyfrowane wiadomości, jednak problemem może być rozpoznanie prawidłowej wiadomości. Jednym ze sposobów jest sprawdzenie stopnia dopasowania częstości liter do języka, w którym została napisana wiadomość.

Częstości liter w tekście napisanym języku polskim jest nastepująca (źródło: [https://commons.wikimedia.org/wiki/File:Polish_letters_frequencies.svg](https://commons.wikimedia.org/wiki/File:Polish_letters_frequencies.svg)):

In [98]:
const letterFrequencies = {
    'a': 8.91,
    'ą': 0.99,
    'b': 1.47,
    'c': 3.96,
    'ć': 0.40,
    'd': 3.25,
    'e': 7.66,
    'ę': 1.11,
    'f': 0.30,
    'g': 1.42,
    'h': 1.08,
    'i': 8.21,
    'j': 2.28,
    'k': 3.51,
    'l': 2.10,
    'ł': 1.82,
    'm': 2.80,
    'n': 5.52,
    'ń': 0.20,
    'o': 7.75,
    'ó': 0.85,
    'p': 3.13,
    'r': 4.69,
    's': 4.32,
    'ś': 1.42,
    't': 3.98,
    'u': 2.50,
    'w': 4.65,
    'y': 3.76,
    'z': 5.64,
    'ź': 0.06,
    'ż': 0.83
  };

W kryptoanalizie przydatna będzie funkcja zlicząjąca częstość występowania liter w podanym tekście.

In [157]:
function getLetterFrequencies(text: string) {
    const output: Record<string, number> = {};
    for (const letter of text) {
        const lowerLetter = letter.toLowerCase();
        output[lowerLetter] = output[lowerLetter] ? output[lowerLetter] + 1 : 1;
    }
    const sum = Object.values(output).reduce((a,b) => a + b, 0);
    for (const letter of Object.keys(output)) {
        output[letter] = output[letter] * 100 / sum;
    }
    return output;
}

Konieczne będzie także znormalizowanie porównywanych częstotliwości tak, aby zawierały te same litery.

In [155]:
function normalizeFrequencies(frequencies1: Record<string, number>, frequencies2: Record<string, number>) {
    const lettersSet1 = new Set(Object.keys(frequencies1));
    const lettersSet2 = new Set(Object.keys(frequencies2));
    const commonLetters: Set<string> = lettersSet1.intersection(lettersSet2);

    let entries1 = Object.entries(frequencies1).filter(([key, value]) => commonLetters.has(key));
    const sum1 = entries1.reduce((a, b) => a + b[1], 0);
    entries1 = entries1.map(([key, value]) => ([key, value/sum1 * 100]))
    const result1 = Object.fromEntries(entries1);

    let entries2 = Object.entries(frequencies2).filter(([key, value]) => commonLetters.has(key));
    const sum2 = entries2.reduce((a, b) => a + b[1], 0);
    entries2 = entries2.map(([key, value]) => ([key, value/sum2 * 100]))
    const result2 = Object.fromEntries(entries2);

    return [result1, result2];
}

Do sprawdzenia w jakim stopniu rozkład częstości w próbie odpowiada rozkładowi oczekiwanemu można użyć testu zgodności chi-kwadrat. Im większa różnica, tym większa będzie wartość chi-kwadrat.

In [161]:
function chiSquaredTest(measuredFrequencies: Record<string, number>, expectedFrequencies: Record<string, number>) {
    let result = 0;
    Object.entries(measuredFrequencies).forEach(([letter, measuredValue]) => {
        const expectedValue = expectedFrequencies[letter];
        if (expectedValue == undefined) return;
        result += (((measuredValue - expectedValue) ** 2) / expectedValue);
    })
    return result
};

chiSquaredTest(getLetterFrequencies("W kryptoanalizie przydatna będzie funkcja zlicząjąca częstotliwość liter w podanym tekście."),letterFrequencies)

18.992119645432787

Teraz mamy wszystko, aby napisać funkcję łamiącą wiadomość metodą brute-force.

In [203]:
function crackBruteForce(encryptedText: string) {
    const t0 = performance.now()

    const crackedCandidates: string[] = [];
    for( let i=0; i < ALPHABET.length; i++) {
        crackedCandidates.push(decrypt(encryptedText, i))
    }

    const testResults = crackedCandidates.map(cracked => {
        const crackedLettersFrequencies = getLetterFrequencies(cracked);
        const [normalizedCrackedLettersFrequencies, normalizedLetterFrequencies] = normalizeFrequencies(crackedLettersFrequencies, letterFrequencies);
        return chiSquaredTest(normalizedCrackedLettersFrequencies, normalizedLetterFrequencies);
    })

    let min = testResults[0];
    let minIndex = 0;

    testResults.forEach((result, index) => {
        if (result < min) {
            min = result;
            minIndex = index;
        }
    })

    const t1 = performance.now()

    return {
        originalMessage: crackedCandidates[minIndex],
        key: minIndex,
        testResult: testResults[minIndex],
        time: t1 - t0,
    }
}

In [206]:
const input = `Ale muszę wam wytłumaczyć, jak narodziła się ta błędna koncepcja denuncjacji przyjemności i chwalebnego bólu, a ja dam wam kompletną relację z systemu i objaśnię prawdziwe nauki wielkiego odkrywcy prawdy, mistrza-budowniczego ludzkiego szczęścia. Nikt nie odrzuca, nie lubi lub unika przyjemności samej w sobie, ponieważ jest to przyjemność, ale dlatego, że ci, którzy nie wiedzą, jak dążyć do przyjemności, racjonalnie napotykają konsekwencje, które są niezwykle bolesne. I znowu nie ma kogoś, kto kocha lub dąży albo pragnie osiągnąć ból sam z siebie, ponieważ jest to ból, ale czasami pojawiają się okoliczności, w których trud i ból mogą przynieść mu wielką przyjemność. Aby wziąć trywialny przykład, który z nas kiedykolwiek podejmuje żmudne ćwiczenia fizyczne, z wyjątkiem uzyskania jakiejś korzyści z tego? Ale kto ma prawo do popełnienia błędu u człowieka, który decyduje się cieszyć się przyjemnością, która nie ma denerwujących konsekwencji, lub tym, który unika bólu, który nie przynosi oczekiwanej przyjemności?
Z drugiej strony, p`;

crackBruteForce(encrypt(input, 55));

{
  originalMessage: "Ale muszę wam wytłumaczyć, jak narodziła się ta błędna koncepcja denuncjacji przyjemności i chwalebnego bólu, a ja dam wam kompletną relację z systemu i objaśnię prawdziwe nauki wielkiego odkrywcy prawdy, mistrza-budowniczego ludzkiego szczęścia. Nikt nie odrzuca, nie lubi lub unika przyjemności samej w sobie, ponieważ jest to przyjemność, ale dlatego, że ci, którzy nie wiedzą, jak dążyć do przyjemności, racjonalnie napotykają konsekwencje, które są niezwykle bolesne. I znowu nie ma kogoś, kto kocha lub dąży albo pragnie osiągnąć ból sam z siebie, ponieważ jest to ból, ale czasami pojawiają się okoliczności, w których trud i ból mogą przynieść mu wielką przyjemność. Aby wziąć trywialny przykład, który z nas kiedykolwiek podejmuje żmudne ćwiczenia fizyczne, z wyjątkiem uzyskania jakiejś korzyści z tego? Ale kto ma prawo do popełnienia błędu u człowieka, który decyduje się cieszyć się przyjemnością, która nie ma denerwujących konsekwencji, lub tym, który unika bólu,